In [1]:
%pip install tqdm
import os
from pathlib import Path
import cv2
import numpy as np
from PIL import Image
import shutil
from tqdm import tqdm


Note: you may need to restart the kernel to use updated packages.


In [2]:
from pathlib import Path

# Detect project root: if we are inside "notebooks", go one level up
CWD = Path.cwd()
if CWD.name == "notebooks":
    PROJECT_ROOT = CWD.parent
else:
    PROJECT_ROOT = CWD

print("Current working dir:", CWD)
print("Project root:", PROJECT_ROOT)

RAW_DATA = PROJECT_ROOT / "data" / "classification_dataset"
PROCESSED_DATA = PROJECT_ROOT / "data" / "processed"

print("Raw dataset:", RAW_DATA)
print("Processed dataset:", PROCESSED_DATA)
print("RAW_DATA exists? →", RAW_DATA.exists())


Current working dir: f:\Aerial_Object_Classification_Detection\notebooks
Project root: f:\Aerial_Object_Classification_Detection
Raw dataset: f:\Aerial_Object_Classification_Detection\data\classification_dataset
Processed dataset: f:\Aerial_Object_Classification_Detection\data\processed
RAW_DATA exists? → True


In [3]:
for split in ["train", "valid", "test"]:
    (PROCESSED_DATA / split / "bird").mkdir(parents=True, exist_ok=True)
    (PROCESSED_DATA / split / "drone").mkdir(parents=True, exist_ok=True)

print("Processed folders ready.")


Processed folders ready.


In [4]:
def preprocess_image(img_path, size=(224,224)):
    img = Image.open(img_path).convert("RGB")
    img = img.resize(size)
    return np.array(img)


In [5]:
image_exts = {".jpg", ".jpeg", ".png", ".bmp"}

for split in ["train", "valid", "test"]:
    split_path = RAW_DATA / split
    if not split_path.exists():
        print(f"[WARNING] Split folder not found: {split_path}")
        continue

    print(f"\nProcessing Split: {split}")

    for cls in os.listdir(split_path):
        src_dir = split_path / cls
        dest_dir = PROCESSED_DATA / split / cls
        dest_dir.mkdir(parents=True, exist_ok=True)

        print(f"   Class: {cls}")

        for img_file in tqdm(src_dir.iterdir()):
            if img_file.suffix.lower() not in image_exts:
                continue

            img = preprocess_image(img_file)
            Image.fromarray(img).save(dest_dir / img_file.name)



Processing Split: train
   Class: bird


1414it [01:16, 18.39it/s]


   Class: drone


1248it [01:02, 20.01it/s]



Processing Split: valid
   Class: bird


...

225it [00:11, 20.03it/s]



Processing Split: test
   Class: bird


121it [00:06, 18.68it/s]


   Class: drone


94it [00:05, 18.09it/s]


In [6]:
import random

label_map = {"bird": 0, "drone": 1}
image_exts = {".jpg", ".jpeg", ".png", ".bmp"}

def inspect_split(split, num_samples=10):
    X, y = [], []
    for cls, label in label_map.items():
        folder = PROCESSED_DATA / split / cls
        if not folder.exists():
            print(f"[WARN] Folder not found: {folder}")
            continue

        files = [f for f in folder.iterdir() if f.suffix.lower() in image_exts]
        if not files:
            print(f"[WARN] No images found in: {folder}")
            continue

        random.shuffle(files)
        sample_files = files[:num_samples]

        for img_file in sample_files:
            img = Image.open(img_file).convert("RGB")
            img = img.resize((224, 224))
            # float32 to save memory
            img_arr = np.array(img, dtype=np.float32) / 255.0
            X.append(img_arr)
            y.append(label)

    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.int64)
    print(f"{split}: X shape = {X.shape}, y shape = {y.shape}")
    return X, y

X_train_small, y_train_small = inspect_split("train", num_samples=10)
X_val_small, y_val_small = inspect_split("valid", num_samples=10)
X_test_small, y_test_small = inspect_split("test", num_samples=10)


train: X shape = (20, 224, 224, 3), y shape = (20,)
valid: X shape = (20, 224, 224, 3), y shape = (20,)
test: X shape = (20, 224, 224, 3), y shape = (20,)


In [7]:
# Load full dataset from processed folders
def load_split(split):
    X, y = [], []
    for cls, label in label_map.items():
        folder = PROCESSED_DATA / split / cls
        if not folder.exists():
            print(f"[WARN] Folder not found: {folder}")
            continue

        files = [f for f in folder.iterdir() if f.suffix.lower() in image_exts]
        if not files:
            print(f"[WARN] No images found in: {folder}")
            continue

        for img_file in tqdm(files, desc=f"Loading {split}/{cls}"):
            img = Image.open(img_file).convert("RGB")
            img = img.resize((224, 224))
            img_arr = np.array(img, dtype=np.float32) / 255.0
            X.append(img_arr)
            y.append(label)

    return np.array(X, dtype=np.float32), np.array(y, dtype=np.int64)

print("Loading full dataset...")
X_train, y_train = load_split("train")
X_val, y_val = load_split("valid")
X_test, y_test = load_split("test")

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

np.savez(PROCESSED_DATA / "classification_data.npz",
         X_train=X_train,
         y_train=y_train,
         X_val=X_val,
         y_val=y_val,
         X_test=X_test,
         y_test=y_test)

print("Saved → classification_data.npz")


Loading full dataset...


Loading train/bird:   0%|          | 0/1414 [00:00<?, ?it/s]

Loading test/drone: 100%|██████████| 94/94 [00:02<00:00, 45.19it/s]


X_train: (2662, 224, 224, 3), y_train: (2662,)
X_val: (442, 224, 224, 3), y_val: (442,)
X_test: (215, 224, 224, 3), y_test: (215,)
Saved → classification_data.npz
